
xgboost 는 GPU 사용 가능하다! [URL](https://xgboost.readthedocs.io/en/latest/gpu/)


*  [xgboost bayes opt](https://www.kaggle.com/paso84/xgboost-bayesian-optimization)

*  [xgboost Bayesian Optimization](https://www.kaggle.com/nanomathias/bayesian-optimization-of-xgboost-lb-0-9769)

*  [XGboost API](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

* [Scikit-Optimize API](https://scikit-optimize.github.io/)

* [XGBClassifer tree_method 설명](https://stackoverflow.com/questions/44058786/specifying-tree-method-param-for-xgboost-in-python)
* [xgboost parameter 설명 모음집](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst)

In [56]:
import pandas as pd , numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.combine import SMOTEENN
from sklearn.metrics import balanced_accuracy_score
from collections import Counter
from MulticoreTSNE import MulticoreTSNE as TSNE
from IPython.display import clear_output
import collections, os
import pickle
from sklearn.model_selection import cross_val_score

from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold


import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb

In [57]:
df = pd.read_csv("./Data/dont-overfit-ii/train.csv")
df.shape

test_df = pd.read_csv("./Data/dont-overfit-ii/test.csv")

test_df = test_df.drop("id", axis = 1)

label = df["target"]
Input = df.drop(columns=['id', 'target'])

test_df = test_df.values.astype(np.float32)

x_data = Input.values.astype(np.float32)
y_datalabel = label
y_data = LabelEncoder().fit_transform(label)
onehot = np.zeros((y_data.shape[0], np.unique(y_data).shape[0]))
for i in range(y_data.shape[0]):
    onehot[i, y_data[i]] = 1.0


    
x_train, x_test, y_train, y_test, y_train_label, y_test_label = train_test_split(x_data, onehot, y_data, test_size = 0.3)

In [76]:
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBClassifier(
        n_jobs = 10 ,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method="approx" , 
        #tree_method='gpu_hist', predictor='gpu_predictor' bayesian과 같이 쓰니 안되는 듯
    ),
    ### Parameter 바꾸는 곳 
    search_spaces = {
        'learning_rate': (0.001, 0.1 , 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': (50, 200),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },
    ## Scoring!
    scoring = 'roc_auc',
    ## CV 할 떄도 StratifiedKFold!!!
    cv = StratifiedKFold(
        n_splits=10,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 10 ,
    n_iter = 10000,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")


In [ ]:
result = bayes_cv_tuner.fit(x_train , y_train_label.values , callback=status_print)

Model #1
Best ROC-AUC: 0.5
Best params: {'reg_lambda': 0.05936070635912049, 'subsample': 0.13556548021189216, 'max_depth': 21, 'scale_pos_weight': 0.060830282487222144, 'learning_rate': 0.0042815319280763466, 'reg_alpha': 5.497557739289786e-07, 'min_child_weight': 2, 'max_delta_step': 13, 'colsample_bylevel': 0.4160029192647807, 'colsample_bytree': 0.7304484857455519, 'gamma': 0.13031389926541354, 'n_estimators': 161}

Model #2
Best ROC-AUC: 0.6502
Best params: {'reg_lambda': 276.5424475574225, 'subsample': 0.9923710598637134, 'colsample_bytree': 0.8844821246070537, 'scale_pos_weight': 0.3016410771843142, 'learning_rate': 0.07988179462781242, 'reg_alpha': 0.0005266983003701547, 'max_depth': 3, 'max_delta_step': 17, 'colsample_bylevel': 0.8390144719977516, 'min_child_weight': 1, 'n_estimators': 103, 'gamma': 4.358684608480795e-07}

Model #3
Best ROC-AUC: 0.7004
Best params: {'reg_lambda': 476.96194787286544, 'subsample': 0.387658500562527, 'min_child_weight': 1, 'reg_alpha': 0.000103768

Model #21
Best ROC-AUC: 0.7464
Best params: {'reg_lambda': 0.0048879464985534336, 'subsample': 0.5689543694097536, 'max_depth': 19, 'reg_alpha': 0.011683028450342707, 'learning_rate': 0.00238149998729586, 'scale_pos_weight': 0.13267482411031659, 'min_child_weight': 2, 'max_delta_step': 16, 'colsample_bylevel': 0.7366877378057127, 'colsample_bytree': 0.9399760402267441, 'n_estimators': 130, 'gamma': 2.6498051478267012e-08}

Model #22
Best ROC-AUC: 0.7464
Best params: {'reg_lambda': 0.0048879464985534336, 'subsample': 0.5689543694097536, 'max_depth': 19, 'reg_alpha': 0.011683028450342707, 'learning_rate': 0.00238149998729586, 'scale_pos_weight': 0.13267482411031659, 'min_child_weight': 2, 'max_delta_step': 16, 'colsample_bylevel': 0.7366877378057127, 'colsample_bytree': 0.9399760402267441, 'n_estimators': 130, 'gamma': 2.6498051478267012e-08}

Model #23
Best ROC-AUC: 0.75
Best params: {'reg_lambda': 0.00022063798523252993, 'subsample': 0.8313737878291801, 'max_depth': 36, 'scale_pos_weig

Model #41
Best ROC-AUC: 0.7634
Best params: {'reg_lambda': 0.005793759077908632, 'subsample': 0.7011979953695395, 'colsample_bytree': 0.5759125142651095, 'scale_pos_weight': 0.06702978662250401, 'learning_rate': 0.07031773600273923, 'reg_alpha': 2.6406372821018985e-08, 'max_depth': 29, 'max_delta_step': 16, 'colsample_bylevel': 0.21539640428051746, 'min_child_weight': 2, 'n_estimators': 200, 'gamma': 1e-09}

Model #42
Best ROC-AUC: 0.7634
Best params: {'reg_lambda': 0.005793759077908632, 'subsample': 0.7011979953695395, 'colsample_bytree': 0.5759125142651095, 'scale_pos_weight': 0.06702978662250401, 'learning_rate': 0.07031773600273923, 'reg_alpha': 2.6406372821018985e-08, 'max_depth': 29, 'max_delta_step': 16, 'colsample_bylevel': 0.21539640428051746, 'min_child_weight': 2, 'n_estimators': 200, 'gamma': 1e-09}

Model #43
Best ROC-AUC: 0.7634
Best params: {'reg_lambda': 0.005793759077908632, 'subsample': 0.7011979953695395, 'colsample_bytree': 0.5759125142651095, 'scale_pos_weight': 0.

Model #61
Best ROC-AUC: 0.7704
Best params: {'reg_lambda': 0.027419762068236424, 'subsample': 0.5411042159119421, 'colsample_bytree': 0.3427354277765229, 'scale_pos_weight': 0.5602936800236206, 'learning_rate': 0.0033262542384601074, 'reg_alpha': 1.890980466816029e-08, 'max_depth': 24, 'max_delta_step': 12, 'colsample_bylevel': 0.4975286998758538, 'min_child_weight': 3, 'n_estimators': 200, 'gamma': 2.541777667298616e-07}

Model #62
Best ROC-AUC: 0.7704
Best params: {'reg_lambda': 0.027419762068236424, 'subsample': 0.5411042159119421, 'colsample_bytree': 0.3427354277765229, 'scale_pos_weight': 0.5602936800236206, 'learning_rate': 0.0033262542384601074, 'reg_alpha': 1.890980466816029e-08, 'max_depth': 24, 'max_delta_step': 12, 'colsample_bylevel': 0.4975286998758538, 'min_child_weight': 3, 'n_estimators': 200, 'gamma': 2.541777667298616e-07}

Model #63
Best ROC-AUC: 0.7704
Best params: {'reg_lambda': 0.027419762068236424, 'subsample': 0.5411042159119421, 'colsample_bytree': 0.3427354277

Model #81
Best ROC-AUC: 0.7704
Best params: {'reg_lambda': 0.027419762068236424, 'subsample': 0.5411042159119421, 'colsample_bytree': 0.3427354277765229, 'scale_pos_weight': 0.5602936800236206, 'learning_rate': 0.0033262542384601074, 'reg_alpha': 1.890980466816029e-08, 'max_depth': 24, 'max_delta_step': 12, 'colsample_bylevel': 0.4975286998758538, 'min_child_weight': 3, 'n_estimators': 200, 'gamma': 2.541777667298616e-07}

Model #82
Best ROC-AUC: 0.7704
Best params: {'reg_lambda': 0.027419762068236424, 'subsample': 0.5411042159119421, 'colsample_bytree': 0.3427354277765229, 'scale_pos_weight': 0.5602936800236206, 'learning_rate': 0.0033262542384601074, 'reg_alpha': 1.890980466816029e-08, 'max_depth': 24, 'max_delta_step': 12, 'colsample_bylevel': 0.4975286998758538, 'min_child_weight': 3, 'n_estimators': 200, 'gamma': 2.541777667298616e-07}

Model #83
Best ROC-AUC: 0.7704
Best params: {'reg_lambda': 0.027419762068236424, 'subsample': 0.5411042159119421, 'colsample_bytree': 0.3427354277